In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [1]:
! pip install lxml

print('Libraries imported.')

Libraries imported.


In [4]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

Get wiki page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M data

In [5]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

In [8]:
wiki_data = soup.find('table',{'class':'wikitable sortable'})
#wiki_data

Crate empty dataframe to fill with wiki_data

In [18]:
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
df1 = pd.DataFrame(columns=column_names)
print("shape",df1.shape)
df1

shape (0, 3)


,PostalCode,Borough,Neighborhood


Read table from url and assume it is the first and only table in the url

In [19]:
wiki_data = soup.find('table',{'class':'wikitable sortable'})
table = wiki_data
rows = table.find_all("tr")
for row in rows:
        columns = row.find_all("td")
        headers = row.find_all("th")
        if len(columns) == 0 : continue
        Postcode = columns[0].text
        Borough = columns[1].text
        Neighbourhood = columns[2].text#.split("/a")
        df1 = df1.append({'PostalCode': Postcode,'Borough': Borough,'Neighborhood': Neighbourhood}, ignore_index=True)
#clean table       
df1 = df1.replace(r'\n','', regex=True) 
df1.drop(df1.index[0],inplace=True)

print("shape",df1.shape,"type",type(df1))
df1.head()

shape (288, 3) type <class 'pandas.core.frame.DataFrame'>


,PostalCode,Borough,Neighborhood
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [20]:
df1=df1[df1.Borough != 'Not assigned']
print("shape",df1.shape)
df1.head()

shape (212, 3)


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


More than one neighborhood can exist in one postal code area. 
Group rows with duplicate PostalCode The Neighborhoods will be separated with a comma.

In [29]:
df2 = df1.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
print('shape',df2.shape)
df2.head()

shape (103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Confirm Postalcode M5A appears in a single row

In [24]:
df2[df2['PostalCode']== 'M5A']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


Not assigned neighborhood cell will be the same as the borough cell

In [25]:
df2['Neighborhood'] = df2.apply(
    lambda row: row['Borough'] if (row['Neighborhood']== 'Not assigned') else row['Neighborhood'],
    axis=1
)
df2[df2['PostalCode']== 'M7A']

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


In [26]:
df2.shape

(103, 3)